In [1]:
######################################################
### Attaching and downloading necessary packagaes ####
######################################################
# if (!requireNamespace("BiocManager", quietly = FALSE))
#     install.packages("BiocManager")
# BiocManager::version()
# library("devtools")
# devtools::install_github("broadinstitute/infercnv", force=TRUE)
# detach("package:infercnv", unload=T)  ### package infercnv_1.15.3  seems to work fine
library(infercnv)
#### loading packages and others 
library("Seurat")
library("tidyverse")
#remotes::install_github("mojaveazure/seurat-disk")
library("SeuratDisk")
#install.packages("ggplot2")
library(ggplot2)
#install.packages("remotes")
#remotes::install_github("chris-mcginnis-ucsf/DoubletFinder", force=TRUE)
library(DoubletFinder)
library(dplyr)
library(tidyverse)

Attaching SeuratObject

-- Attaching core tidyverse packages ----------------------------------------------------------------------------------------------------------------------------------------------- tidyverse 2.0.0 --
v dplyr     1.1.1     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ----------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [2]:
#Below is the code for cnv score calculation after running inferCNV:
# Read the infercnv result tableinto a matrix format
expr <- read.table(paste0("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/infercnv_result_1.10.1/infercnv.observations.txt"), header=T) %>% as.matrix()
# Scale the expression matrix by subtracting the column means and dividing by the column standard deviations
expr.scale <- scale(t(expr))
# Subtract the minimum value of each column from the scaled expression matrix
tmp1 <- sweep(expr.scale, 2, apply(expr.scale, 2, min),'-')
# Calculate the range (maximum - minimum) of each column in the scaled expression matrix
tmp2 <- apply(expr.scale, 2, max) - apply(expr.scale,2,min)
# Normalize the scaled expression matrix to the range of -1 to 1
expr_1 <- t(2*sweep(tmp1, 2, tmp2, "/")-1)
# Calculate the CNV score by summing the squared values of each column in the normalized expression matrix
cnv_score <- as.data.frame(colSums(expr_1 * expr_1))
#head(cnv_score)
#### CNV score calculation ends here #######

###################################
### CNV plot starts from here #####
###################################
### before plotting, needs some sort of processing such as doing add rownames of cell ID to dataframe and assign name CellID to this
cnv_score <- tibble::rownames_to_column(cnv_score, var = "CellID")

## change name of second column to score, initially it is "colSums(expr_1 * expr_1)"
colnames(cnv_score)[2] ="score"
# saving this dataframe for future use, if needed
write.table(cnv_score, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/infercnv_result_1.10.1/cnv_score.txt", sep = "\t", row.names=FALSE)

## Since, the tow dataframe i.e. cnv score dataset and annotation dataset has similar thing in terms of cellID but in different order
## so, make them in similar order
# Read File 1 into a data frame
file1 <- read.table("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/infercnv_result_1.10.1/cnv_score.txt", header = TRUE)

# Read File 2 into a data frame
file2 <- read.table("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/annotation.txt", header = TRUE)

### make copy of the annotation file, since InferCNV needs the file without header and here we are adding header for plot and so
file2_cp <- read.table("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/annotation.txt", header = FALSE)
# Add headers to the data
headers <- c("CellID", "Info")
colnames(file2_cp) <- headers

# save the annotation copied file with header info
#write.table(file2_cp, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/annotation_cp.txt", sep = "\t", col.names = TRUE, row.names = FALSE)
## since the name of cell ID in file2_cp and file1 are different, in file 1 name is separated by "-" while in file2_cp it is by "."
file3 <- read.table("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/annotation_cp.txt", header = TRUE)
# Merge File 1 and File 2 based on the cell ID column
merged <- merge(file1, file3, by = "CellID")

# Reorder the rows of File 1 based on the order in File 2
ordered_file1 <- merged[order(merged$CellID), ]

## save the dataframe for future use 
write.table(ordered_file1, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/cnv_score_ordered_annotation.txt", sep = "\t", row.names = FALSE)

### plotting the results ###
library(ggplot2)
png("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/SCAF3163_classic.jpg", width = 3000, height = 2000, res=300)
ggplot(ordered_file1, aes(x=Info, y=score, fill=Info)) + 
  geom_violin(trim=FALSE)+
  geom_boxplot(width=0.1, fill="white")+
  labs(title="",x="Cluster", y = "CNV Scores") +  theme_classic() + theme(axis.text.x = element_text(angle = 30, hjust = 1)) 
dev.off()

png("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/SCAF3163_minimal.jpg", width = 3000, height = 2000, res=300)
ggplot(ordered_file1, aes(x=Info, y=score, fill=Info)) + 
  geom_violin(trim=FALSE)+
  geom_boxplot(width=0.1, fill="white")+
  labs(title="",x="Cluster", y = "CNV Scores") + theme_minimal() + theme(axis.text.x=element_text(angle=30, hjust=1))
dev.off()

png 
  2

png 
  2

In [7]:
file3 <- read.table("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/annotation_cp.txt", header = TRUE)
# Merge File 1 and File 2 based on the cell ID column
merged <- merge(file1, file3, by = "CellID")

# Reorder the rows of File 1 based on the order in File 2
ordered_file1 <- merged[order(merged$CellID), ]

## save the dataframe for future use 
write.table(ordered_file1, file = "/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/cnv_score_ordered_annotation.txt", sep = "\t", row.names = FALSE)

### plotting the results ###
library(ggplot2)
png("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/SCAF3163_classic.jpg", width = 3000, height = 2000, res=300)
ggplot(ordered_file1, aes(x=Info, y=score, fill=Info)) + 
  geom_violin(trim=FALSE)+
  geom_boxplot(width=0.1, fill="white")+
  labs(title="",x="Cluster", y = "CNV Scores") +  theme_classic() + theme(axis.text.x = element_text(angle = 30, hjust = 1)) 
dev.off()

png("/data/kumarr9/scRNA/scRNA_results/cell_cycle_regress_out/SCAF3163/alternate/SCAF3163_minimal.jpg", width = 3000, height = 2000, res=300)
ggplot(ordered_file1, aes(x=Info, y=score, fill=Info)) + 
  geom_violin(trim=FALSE)+
  geom_boxplot(width=0.1, fill="white")+
  labs(title="",x="Cluster", y = "CNV Scores") + theme_minimal() + theme(axis.text.x=element_text(angle=30, hjust=1))
dev.off()

png 
  2

png 
  2